# Serialize/Deserialize

The `EODataAccessGateway` class provides methods to save a `SearchResult` object to a GeoJSON file or to load a GeoJSON file to a `SearchResult`. These methods come in handy to save the state of a search and restore it later.

The credentials are required in this notebook to download a quicklook from *PEPS*.

In [1]:
import os

os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

In [2]:
from eodag import EODataAccessGateway
from eodag import setup_logging

setup_logging(1)

dag = EODataAccessGateway()
dag.set_preferred_provider("peps")

2021-04-12 23:54:53,617-15s eodag.config                     [INFO    ] Loading user configuration from: /home/maxime/.config/eodag/eodag.yml
2021-04-12 23:54:54,450-15s eodag.core                       [INFO    ] Locations configuration loaded from /home/maxime/.config/eodag/locations.yml


## Serialize

The method [serialize](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.serialize) allows to save a `SearchResult` as a *GeoJSON* file.

In [3]:
search_results, _ = dag.search(
    productType="S2_MSI_L1C",
    start="2021-03-01",
    end="2021-03-31",
    geom={"lonmin": 1, "latmin": 43, "lonmax": 2, "latmax": 44},
    items_per_page=2
)

2021-04-12 23:54:56,405-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-04-12 23:54:56,411-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2021-03-01&completionDate=2021-03-31&geometry=POLYGON ((1.0000 43.0000, 1.0000 44.0000, 2.0000 44.0000, 2.0000 43.0000, 1.0000 43.0000))&productType=S2MSI1C&maxRecords=1&page=1
2021-04-12 23:54:57,131-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2021-03-01&completionDate=2021-03-31&geometry=POLYGON ((1.0000 43.0000, 1.0000 44.0000, 2.0000 44.0000, 2.0000 43.0000, 1.0000 43.0000))&productType=S2MSI1C&maxRecords=2&page=1
2021-04-12 23:54:57,983-15s eodag.core                       [INFO    ] Found 48 result(s) on provider 'peps'


In [4]:
search_results

[EOProduct(id=S2B_MSIL1C_20210328T103629_N0209_R008_T31TCJ_20210328T124650, provider=peps), EOProduct(id=S2B_MSIL1C_20210328T103629_N0209_R008_T31TCH_20210328T124650, provider=peps)]

A folder is created to save the output of this notebook.

In [5]:
workspace = "eodag_workspace_serialize_deserialize"
if not os.path.isdir(workspace):
    os.mkdir(workspace)

In [6]:
output_file = os.path.join(workspace, "search_results.geojson")

dag.serialize(
    search_results, 
    filename=output_file
)

'eodag_workspace_serialize_deserialize/search_results.geojson'

## Deserialize

There are two methods offered by `EODataAccessGateway` to load a search result saved as a GeoJSON:

* [deserialize](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize): it  simply recreates a `SearchResult` and the `EOProduct` it contained
* [deserialize_and_register](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize_and_register): it also recreates a `SearchResult` but additionally registers for each `EOProduct` the information it requires to download itself

In [7]:
deserialized_search_results = dag.deserialize(output_file)
deserialized_search_results

[EOProduct(id=S2B_MSIL1C_20210328T103629_N0209_R008_T31TCJ_20210328T124650, provider=peps), EOProduct(id=S2B_MSIL1C_20210328T103629_N0209_R008_T31TCH_20210328T124650, provider=peps)]

In [8]:
deserialized_search_results[0].get_quicklook()

AttributeError: 'NoneType' object has no attribute 'config'

Trying to download a quicklook from a `SearchResult` that was loaded with [deserialize](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize) since it doesn't try to configure each product so that it can be downloaded.

In [9]:
deserialized_and_registered = dag.deserialize_and_register(output_file)
deserialized_and_registered

[EOProduct(id=S2B_MSIL1C_20210328T103629_N0209_R008_T31TCJ_20210328T124650, provider=peps), EOProduct(id=S2B_MSIL1C_20210328T103629_N0209_R008_T31TCH_20210328T124650, provider=peps)]

In [10]:
from eodag.utils import NotebookProgressCallback

quicklook_path = deserialized_and_registered[0].get_quicklook(
    base_dir=workspace,
    progress_callback=NotebookProgressCallback()
)

2021-04-12 23:55:07,085-15s eodag.api.product                [INFO    ] Download recorded in /home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_serialize_deserialize/S2B_MSIL1C_20210328T103629_N0209_R008_T31TCJ_20210328T124650


Downloading the quicklook with [deserialize_and_register](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize_and_register) works as expected.